In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

pd.pandas.set_option('display.max_columns', None)
%matplotlib inline
random.seed(7)

In [75]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [76]:
src_csv = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data', 'concrete.csv'))
df = pd.read_csv(src_csv)

df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Concrete compressive strength
0,2.477915,-0.858192,-0.847144,-0.954349,-0.634503,0.863154,-1.227306,-0.153159,2.645408
1,2.477915,-0.858192,-0.847144,-0.954349,-0.634503,1.056164,-1.227306,-0.153159,1.561421
2,0.491425,0.811541,-0.847144,2.250592,-1.091166,-0.526517,-2.269700,-0.153159,0.266627
3,0.491425,0.811541,-0.847144,2.250592,-1.091166,-0.526517,-2.269700,-0.153159,0.313340
4,-0.790459,0.693195,-0.847144,0.502442,-1.091166,0.070527,0.673158,-0.153159,0.507979


In [77]:
X_data = df.drop(df.columns[-1], axis=1).values
y_data = df[df.columns[-1]].values.reshape(-1, 1)
print(X_data.shape, y_data.shape)

(1030, 8) (1030, 1)


In [78]:
# train, validation, test = 60%, 20%, 20%
X_tv, X_test, y_tv, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25, random_state=7)

In [79]:
# model = Lasso(alpha=0.1)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# 
# mse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# 
# print(mae)
# print(model.intercept_)
# print(model.coef_)

In [80]:
# class MyLassoRegression_BGD:
#     def __init__(self, learning_rate=0.01, lambda_=0.1, epochs=1000):
#         self.learning_rate = learning_rate
#         self.lambda_ = lambda_
#         self.epochs = epochs
#         self.theta = None
#         
#     def fit(self, X, y):
#         X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
#         n_samples, n_dims = X.shape
#               
#         self.theta = np.zeros((n_dims, 1))
#         I = np.identity(n_dims)
#         I[0, 0] = 0  
#         
#         cost_list = []
#         epoch_list = []
#     
#         for epoch in range(self.epochs):
#             y_pred = X @ self.theta
#             
#             dt = (2 / n_samples) * (X.T @ (y_pred - y)) + 2 * self.lambda_ * np.sign(self.theta) * I.diagonal().reshape(-1, 1)
#     
#             self.theta -= self.learning_rate * dt
#             
#             if epoch % 10 == 0:
#                 cost = np.mean(np.square(y_pred - y)) + self.lambda_ * np.sum(self.theta[1:])
#                 cost_list.append(cost)
#                 epoch_list.append(epoch)
#             
#         plt.plot(epoch_list, cost_list)
#         plt.xlabel('Epoch')
#         plt.ylabel('Cost')
#         plt.show()
#     
#     def predict(self, X):
#         return X @ self.theta[1:] + self.theta[0]
#     
#     
# model = MyLassoRegression_BGD()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# 
# mse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# 
# print(mae)
# print(model.theta)

In [81]:
from cvxopt import solvers, matrix
from sklearn.metrics import mean_squared_error

class MySVR:    
    def __init__(self, kernel='linear', degree=3, C=10, eps=0.01):
        self.kernel = kernel
        self.degree = degree
        self.C = C
        self.eps = eps
        self.kernel_matrix = None
        self.alpha_diff = None
        self.X = None
        self.b = 0
    
    def linear_kernel(self, x, z):
        return x @ z.T

    def poly_kernel(self, x, z, degree, intercept):
        return np.power((x @ z.T) + intercept, degree)
    
    def rbf_kernel(self, x, z, sigma):
        n = x.shape[0]
        m = z.shape[0]
        xx = np.dot(np.sum(np.power(x, 2), 1).reshape(n, 1), np.ones((1, m)))
        zz = np.dot(np.sum(np.power(z, 2), 1).reshape(m, 1), np.ones((1, n)))
        return np.exp(-(xx + zz.T - 2 * np.dot(x, z.T)) / (2 * sigma ** 2))
    
    def fit(self, X, y):
        self.X = X
        m = X.shape[0]
        print(f"X: {X.shape}")
        print(f"y: {y.shape}")
        
        if self.kernel == 'linear':
            self.kernel_matrix = self.linear_kernel(X, X)
        elif self.kernel == 'poly':
            self.kernel_matrix = self.poly_kernel(X, X, self.degree, 0)
        elif self.kernel == 'rbf':
            self.kernel_matrix = self.rbf_kernel(X, X, 1)
        
        K = self.kernel_matrix
        print(f"K: {K.shape}")
        
        P = matrix(0.5 * np.vstack((np.hstack((K, - K)), np.hstack((- K, K)))), (2 * m, 2 * m), 'd')
        print(f"P: {P.size}")
        
        q = matrix(np.vstack((np.ones((m, 1)) * self.eps - y.reshape(-1, 1), np.ones((m, 1)) * self.eps + y.reshape(-1, 1))), (2 * m, 1), 'd')
        print(f"q: {q.size}")
        
        G = matrix(np.vstack((np.eye(2 * m), - np.eye(2 * m))))
        print(f"G: {G.size}")
        
        h = matrix(np.vstack((np.ones((2 * m, 1)) * self.C, np.zeros((2 * m, 1)))))
        print(f"h: {h.size}")
        
        A = matrix(np.hstack((np.ones((1, m)), - np.ones((1, m)))))
        print(f"A: {A.size}")
        
        b = matrix(0.0)
        print(f"b: {b.size}")
        
        sol = solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol["x"])
        alphas[np.abs(alphas) < 1e-5] = 0
        
        self.alpha_diff = alphas[0:m] - alphas[m:2*m]
        
        for k in range(m):
            if alphas[k] > 0:
                self.b = y[k] - np.sum(self.alpha_diff * X * X[k])
                break
                
    def predict(self, X):
        m = X.shape[0]
        y_pred = np.zeros(m)
        if self.kernel == 'linear':
            for i in range(m):
                y_pred[i] = self.linear_kernel(self.X, X[i, :]).T @ self.alpha_diff + self.b
        elif self.kernel == 'poly':
            for i in range(m):
                y_pred[i] = self.poly_kernel(self.X, X[i, :], self.degree, 0).T @ self.alpha_diff + self.b
        elif self.kernel == 'rbf':
            for i in range(m):
                y_pred[i] = self.rbf_kernel(self.X, X[i, :], 1).T @ self.alpha_diff + self.b
        return y_pred
        
svr = MySVR()
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse

X: (618, 8)
y: (618, 1)
K: (618, 618)
P: (1236, 1236)
q: (1236, 1)
G: (2472, 1236)
h: (2472, 1)
A: (1, 1236)
b: (1, 1)
     pcost       dcost       gap    pres   dres
 0: -1.0406e+02 -2.3807e+04  2e+04  5e-13  2e-13
 1: -7.8599e+02 -4.1325e+03  3e+03  1e-12  2e-13
 2: -1.9901e+03 -2.6596e+03  7e+02  7e-13  2e-13
 3: -2.2891e+03 -2.4360e+03  1e+02  3e-12  2e-13
 4: -2.3439e+03 -2.3960e+03  5e+01  4e-13  2e-13
 5: -2.3602e+03 -2.3827e+03  2e+01  4e-13  2e-13
 6: -2.3673e+03 -2.3767e+03  9e+00  2e-12  2e-13
 7: -2.3711e+03 -2.3733e+03  2e+00  4e-12  2e-13
 8: -2.3719e+03 -2.3726e+03  7e-01  1e-12  2e-13
 9: -2.3722e+03 -2.3723e+03  1e-01  2e-12  2e-13
10: -2.3722e+03 -2.3722e+03  4e-02  1e-12  2e-13
11: -2.3722e+03 -2.3722e+03  3e-03  3e-14  2e-13
12: -2.3722e+03 -2.3722e+03  4e-05  2e-12  2e-13
Optimal solution found.


C:\Users\Balaji\AppData\Local\Temp\ipykernel_23904\1755415229.py:78: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred[i] = self.linear_kernel(self.X, X[i, :]).T @ self.alpha_diff + self.b


4.80631170605686